Outdated

In [ ]:
import mlflow
from mlflow.tracking import MlflowClient

In [ ]:
from mlflow.tracking import MlflowClient

In [3]:
client = MlflowClient(tracking_uri="sqlite:///mlflow.db")

client.search_experiments()

[<Experiment: artifact_location='/workspaces/mlops-zoomcamp/02-expe/mlruns/1', creation_time=1747637206697, experiment_id='1', last_update_time=1747637206697, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1747636010238, experiment_id='0', last_update_time=1747636010238, lifecycle_stage='active', name='Default', tags={}>]

In [10]:
# client.create_experiment(name="my-cool-experiment")


In [16]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='1',
    filter_string="metrics.rmse < 25",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"]
)

In [17]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id: ec3db06f126c4b1b89826b52a72c0e04, rmse: 24.4938
run id: f05e37ce9b9744e19e28da3d717fa230, rmse: 24.6043
run id: 5e21f280e95747bf92e7ca2e0a290d30, rmse: 24.6144
run id: 714edc233ea541fa8ae20034549f23f0, rmse: 24.6197
run id: 2bb5cff5e69948c4a719e2bc64fa21cc, rmse: 24.7323


In [22]:
run_id = "a2f516d97f8848fb990e80aa20f78110"
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name="testtest")

Registered model 'testtest' already exists. Creating a new version of this model...


MissingConfigException: Yaml file '/workspaces/mlops-zoomcamp/02-expe/mlruns/1/a2f516d97f8848fb990e80aa20f78110/meta.yaml' does not exist.

In [24]:
model_name = "testtest"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 2, stage: Staging


/tmp/ipykernel_2697/900954435.py:2: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  latest_versions = client.get_latest_versions(name=model_name)


In [28]:
from datetime import datetime

model_version = 2
new_stage = "Staging"

date = datetime.today().date()
client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_stage} on {date}"
)

<ModelVersion: aliases=[], creation_timestamp=1747791970988, current_stage='Staging', description='The model version 2 was transitioned to Staging on 2025-05-21', last_updated_timestamp=1747812889011, name='testtest', run_id='a2f516d97f8848fb990e80aa20f78110', run_link='', source='/workspaces/mlops-zoomcamp/02-expe/mlruns/1/a2f516d97f8848fb990e80aa20f78110/artifacts/models_mlflow', status='READY', status_message=None, tags={}, user_id=None, version=2>

In [35]:
from sklearn.metrics import mean_squared_error
import pandas as pd


def read_dataframe(filename):
    df = pd.read_parquet(filename).head(90000)

    df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)
    df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df


def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)


def test_model(name, stage, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")
    y_pred = model.predict(X_test)
    return {"rmse": mean_squared_error(y_test, y_pred, squared=False)}

In [30]:
client.download_artifacts(run_id=run_id, path='preprocessor', dst_path='.')

'/workspaces/mlops-zoomcamp/02-expe/preprocessor'

In [31]:
import pickle

with open("preprocessor/preprocessor.b", "rb") as f_in:
    dv = pickle.load(f_in)

In [36]:
df = read_dataframe("https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2025-01.parquet")
X_test = preprocess(df, dv)


In [37]:
target = "duration"
y_test = df[target].values

In [39]:
%time test_model(name=model_name, stage="Production", X_test=X_test, y_test=y_test)


MlflowException: No versions of model with name 'testtest' and stage 'Production' found

In [40]:
%time test_model(name=model_name, stage="Staging", X_test=X_test, y_test=y_test)


MlflowException: No versions of model with name 'testtest' and stage 'Staging' found

In [41]:
client.transition_model_version_stage(
    name=model_name,
    version=4,
    stage="Production",
    archive_existing_versions=True
)

/tmp/ipykernel_2697/3610659809.py:1: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


MlflowException: Model Version (name=testtest, version=4) not found